**Rocks Classifictions**

**Import important library and Function**

In [1]:

import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import cv2

import seaborn as sns
import shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
sns.set_style('darkgrid')
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import Dense,Flatten , Conv2D , MaxPooling2D , Dropout
from tensorflow.keras.models import Model , Sequential
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory

ModuleNotFoundError: No module named 'pandas'

**Define function for deleting Corupted Image**

In [ ]:
def del_corupted_images(filename):
                 del_count = 0     
                 if os.path.isfile(filename):
                    try:
                        fobj = open(filename,'rb')
                        is_JFIF =  b'JFIF' in fobj.peek(10)
                    finally:
                        fobj.close()
                    if not is_JFIF:
                        del_count += 1
                 return del_count

**Delete the Image Repeated Exactly**

In [ ]:
import hashlib

def del_shw_RepIm(path):
            os.chdir(path)
            files_list = os.listdir()
            duplicates = []
            hash_keys = dict()
            for index, filename in  enumerate(os.listdir('.')):  #listdir('.') = current directory
                 if os.path.isfile(filename):
                    with open(filename, 'rb') as f:
                        filehash = hashlib.md5(f.read()).hexdigest()

                    if filehash not in hash_keys: 
                        hash_keys[filehash] = index
                        if del_corupted_images(path+'/'+ filename) != 1 :
                            filepaths.append(path +'/'+ filename)
                            labels.append(lab)
                    else:
                         duplicates.append((index,hash_keys[filehash]))

            for file_indexes in duplicates:
                     try:    
                        
                         plt.subplot(121),plt.imshow(plt.imread(files_list[file_indexes[1]]))
                         plt.title(file_indexes[1]), plt.xticks([]), plt.yticks([])

                         plt.subplot(122),plt.imshow(plt.imread(files_list[file_indexes[0]]))
                         plt.title(str(file_indexes[0]) + ' duplicate'), plt.xticks([]), plt.yticks([])
                         plt.show()

                     except OSError as e:
                        print(error)
                        continue

In [ ]:
path='C:\Users\avane\Desktop\Clubs\Mars Rover\Life Sciences\ML\Model 1\archive\Dataset'
filepaths=[]
labels=[]

for sub_path in os.listdir(path):
        for lab in os.listdir(path+'/'+sub_path):
             del_shw_RepIm(path+'/'+sub_path+'/'+lab) 

In [ ]:
print ('filepaths: ', len(filepaths), '   labels: ', len(labels))
pd.Series(labels).value_counts()

**Delete duplicated images use CNN from imagededup**

In [ ]:
!pip install imagededup
# please note this package - imagededup was custom-installed in this kernel 
from imagededup.methods import CNN
from imagededup.utils import plot_duplicates
import pandas as pd
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = (20, 20)

In [ ]:
cnn = CNN()

In [ ]:
for sub_path in os.listdir(path):
        for lab in os.listdir(path+'/'+sub_path):
                dir=path+'/'+sub_path+'/'+lab
                encodings = cnn.encode_images(image_dir=dir)
                duplicate = cnn.find_duplicates_to_remove(image_dir=dir,min_similarity_threshold=0.9499) 
                for indx , path_img in enumerate(filepaths):
                    if path_img.split('/')[-2] == lab :
                          for  namImg in duplicate :
                                 if dir+'/'+namImg == path_img:
                                    filepaths.pop(indx )
                                    labels.pop(indx )

In [ ]:
print ('filepaths: ', len(filepaths), '   labels: ', len(labels))  

**Save the paths of images in data frame**

In [ ]:
Fseries=pd.Series(filepaths, name='filepaths')
Lseries=pd.Series(labels, name='labels')
df=pd.concat([Fseries,Lseries], axis=1)

balance=df['labels'].value_counts()
print (balance) 

**Create file to save augement Image** 

In [ ]:
pth= '/kaggle/working/new_Igneous'
os.chdir('/kaggle/working/')
if os.path.isfile(pth) == 0:
    os.mkdir(pth)

**Create new foder for 7 classes**

In [ ]:
classes = ['Basalt' ,'Granite' , 'Marble' ,'Quartzite'  , 'Coal'  , 'Sandstone' , 'Limestone']
for cls in classes:
    if os.path.isfile(pth+'/'+cls) == 0 :
        os.mkdir(pth+'/'+cls)

**Delete the content for every classes** 

In [ ]:
# for cls in classes:
#     if os.path.isfile(pth+'/'+cls) == 0 :
#         shutil.rmtree(pth+'/'+cls)

In [ ]:
print ('filepaths: ', len(filepaths), '   labels: ', len(labels)) 

**Copy image after delete Corupted , Repeated and duplicated images**

In [ ]:
import os
import shutil

src_path = '/kaggle/input/rock-classification/Dataset'
dst_path = '/kaggle/working/new_Igneous'



# Move each jpg file to the destination directory
for file in filepaths:
    x = file.split('/')[-2:]
    shutil.copy(file , os.path.join(dst_path,x[0]+'/'+x[1]))


**Load training and validation sets and make augmentation**

In [ ]:
# Load training and validation sets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dst_path,validation_split=0.2,
    subset='training',
    labels='inferred',
    label_mode='categorical',
    image_size=[224, 224],
    seed=123,
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dst_path,validation_split=0.2,
    subset='validation',
    labels='inferred',
    label_mode='categorical',
    image_size=[224, 224],
    seed=123,
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    train_ds
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    valid_ds
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

**Bulid the model**

In [ ]:
model = Sequential()
model.add(Rescaling(1./255 ,input_shape=(224,224,3)))
model.add(Conv2D(16, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(7 , activation = 'softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam' , metrics=['accuracy'])

In [ ]:
# Set callback functions to early stop training 
mycallbacks = [EarlyStopping(monitor='val_loss', patience=5,  restore_best_weights=True)]
hist = model.fit(train_ds,validation_data=valid_ds, epochs=50 , callbacks=mycallbacks )

**Evaluate the model**

In [ ]:
model.evaluate(valid_ds)